In [1]:
import pandas as pd
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import time

data_path = "/run/media/kevin/Volume/OpenImages/"

In [6]:
class ImageDataset(Dataset):
    """Test Class"""
    
    def __init__(self, label_file, root_dir, label_name_path, transform=None):
        self.labels = pd.read_csv(label_file, index_col = 0)
        self.root_dir = root_dir
        self.transform = transform
        self.label_names = pd.read_csv(label_name_path, index_col=0)
        
        self.images = list(set(self.labels.index))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        """ Get an image"""
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        # Load image
        img_id = self.images[idx]
        img_path = os.path.join(self.root_dir, self.images[idx]+".jpg")
        image = Image.open(img_path)
        
        # Create label vector
        image_label = np.array(self.labels.loc[img_id].item().split(",")).astype(int)
        
        # Apply transform
        if self.transform is not None:
            image = self.transform(image)
        
        image = np.asarray(image)
        sample = {'image': image, 'label': image_label}
        
        return sample
    
        

In [7]:
transform = transforms.Compose(
    [transforms.Scale((299, 299)),
     transforms.Grayscale(3),
     transforms.ToTensor()])

root_dir = os.path.join(data_path, "pics")
csv_path = os.path.join(data_path, "open_image_labels_formatted.csv")
label_name_path = os.path.join(data_path, "label_names.csv")
dataset = ImageDataset(label_file = csv_path, root_dir = root_dir,
                       label_name_path = label_name_path, transform=transform)

## Model

In [14]:
n_classes = len(dataset.label_names)
print(f"{n_classes} classes")

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(in_features=512, out_features=n_classes, bias=True)

# specify device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

data_loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=12)
len(data_loader)

601 classes


99520

In [15]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(data_loader):
        print(i)
        # get the inputs; data is a list of [inputs, labels]
        imgs = batch['image'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(imgs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/kevin/.conda/envs/image/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `item` has been deprecated and will be removed in a future version
/hom

0
[1,     1] loss: 0.007132
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


KeyboardInterrupt: 